# National Grid Power Cuts - API Exploration

**Goal:** Fetch and explore live power outage data from National Grid. 
## API Endpoint <br>  
**CKAN DataStore API:** https://connecteddata.nationalgrid.co.uk/api/3/action/

## 0. Setup

In [7]:
# Import Libaries 
import requests
import pandas as pd
import altair as alt
from datetime import datetime
import json

# Enable Altair to display larger datasets
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## 1. API Connection Test

In [8]:
#  Test API with small query
BASE_URL = "https://connecteddata.nationalgrid.co.uk/api/3/action/datastore_search" # API Endpoint
RESOURCE_ID = "292f788f-4339-455b-8cc0-153e14509d4d" # Resource ID for Power Cuts Dataset

# Test the API connection with a small sample
def test_api_connection(limit=1000):
    """
    Test the National Grid API with a small query
    """
    params = {
        'resource_id': RESOURCE_ID,
        'limit': limit
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Raise error for bad status codes

        data = response.json()

        print(f"✅ API Connection Successful!")
        print(f"Status Code: {response.status_code}")
        print(f"\nResponse structure keys: {data.keys()}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"❌ API Connection Failed: {e}")
        return None
    
# Run the test
test_response = test_api_connection(limit=5)

✅ API Connection Successful!
Status Code: 200

Response structure keys: dict_keys(['help', 'success', 'result'])


## 2. Understanding the Data Schema

In [9]:
# Examine the full response structure
if test_response and test_response['success']:
    result = test_response['result']

    print("=== Response Metadata ===")
    print(f"Total records available: {result.get('total', 'Unknown')}")
    print(f"Records returned in this query: {len(result.get('records', []))}")

    # Look at the fields/schema
    if 'fields' in result:
        print("\n=== Available Fields (Schema) ===")
        fields_df = pd.DataFrame(result['fields'])
        print(fields_df)


# Convert to DataFrame for easier exploration
df_sample = pd.DataFrame(test_response['result']['records'])

print("\n=== DataFrame Overview ===")
print(f"Shape: {df_sample.shape}")
print(f"\nData Types:\n{df_sample.dtypes}")

print(f"\nFirst Few Rows:")
display(df_sample.head())

# =======================================================
# Check unique values in Status and Planned columns
print("\n=== Unique Values Analysis ===")
print(f"\nUnique values in 'Status' column:")
print(df_sample['Status'].unique())
print(f"Value counts: {df_sample['Status'].value_counts().to_dict()}")

print(f"\nUnique values in 'Planned' column:")
print(df_sample['Planned'].unique())
print(f"Value counts: {df_sample['Planned'].value_counts().to_dict()}")

# Check data types
print(f"\nData type of 'Status': {df_sample['Status'].dtype}")
print(f"Data type of 'Planned': {df_sample['Planned'].dtype}")

# See some sample combinations
print(f"\nSample Status + Planned combinations:")
print(df_sample[['Status', 'Planned']].drop_duplicates().head(10))

=== Response Metadata ===
Total records available: 40
Records returned in this query: 5

=== Available Fields (Schema) ===
                    id       type
0                  _id        int
1          Upload Date  timestamp
2               Region       text
3          Incident ID       text
4        Confirmed Off    numeric
5        Predicted Off    numeric
6             Restored    numeric
7               Status       text
8              Planned       text
9             Category       text
10     Resource Status       text
11          Start Time  timestamp
12                 ETR  timestamp
13             Voltage       text
14   Location Latitude    numeric
15  Location Longitude    numeric
16           Postcodes       text

=== DataFrame Overview ===
Shape: (5, 17)

Data Types:
_id                     int64
Upload Date            object
Region                 object
Incident ID            object
Confirmed Off           int64
Predicted Off           int64
Restored                int64

,_id,Upload Date,Region,Incident ID,Confirmed Off,Predicted Off,Restored,Status,Planned,Category,Resource Status,Start Time,ETR,Voltage,Location Latitude,Location Longitude,Postcodes
0,1,2025-11-20T11:10:00,South West,INCD-10906-u,0,40,0,Awaiting,false,LV GENERIC,UNASSIGNED,2025-11-20T09:55:00,2025-11-20T15:30:00,LV,50.95478,-2.720655,"TA15 6UR, TA15 6UH, TA15 6UL"
1,2,2025-11-20T11:10:00,South West,INCD-75162-l,83,0,700,In Progress,false,HV GENERIC,ONS,2025-11-20T09:08:00,2025-11-20T11:00:00,HV,49.91380,-6.309997,"TR24 0QQ, TR24 0QE, TR24 0QD, TR24 0AB, TR24 0..."
2,3,2025-11-20T11:10:00,South West,INCD-18791-t,2,0,1,In Progress,false,LV UNDERGROUND,COMP,2025-10-22T08:59:00,None,LV,50.51944,-3.528319,"TQ12 4QS, TQ12 4QT"
3,4,2025-11-20T11:10:00,South West,INCD-75129-l,11,0,29,Awaiting,false,HV OVERHEAD,ASSN,2025-11-19T17:27:00,2025-11-20T12:00:00,HV,49.93407,-6.299540,"TR24 0QQ, TR24 0PX, TR24 0QH"
4,5,2025-11-20T11:10:00,South West,INCD-10903-u,0,39,0,In Progress,false,LV GENERIC,ONS,2025-11-20T09:38:00,None,LV,51.54241,-2.622230,"BS35 5RA, BS35 5SF, BS35 5RD, BS35 5RE, BS35 5RU"



=== Unique Values Analysis ===

Unique values in 'Status' column:
['Awaiting' 'In Progress']
Value counts: {'In Progress': 3, 'Awaiting': 2}

Unique values in 'Planned' column:
['false']
Value counts: {'false': 5}

Data type of 'Status': object
Data type of 'Planned': object

Sample Status + Planned combinations:
        Status Planned
0     Awaiting   false
1  In Progress   false
